# Classification_of_Consumer_Complaints(Japanese) by Llama3-70B 20240504

## 1. ライブラリをインポート (Import library)

In [ ]:
!pip -q install langchain-groq duckduckgo-search
!pip -q install -U langchain_community tiktoken langchainhub
!pip -q install -U langchain langgraph tavily-python

In [ ]:
!pip show langchain

Name: langchain
Version: 0.1.17
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: aiohttp, async-timeout, dataclasses-json, jsonpatch, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 


In [ ]:
!pip show langgraph

Name: langgraph
Version: 0.0.44
Summary: langgraph
Home-page: https://www.github.com/langchain-ai/langgraph
Author: 
Author-email: 
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: langchain-core
Required-by: 


In [ ]:
!pip -q install langchain tiktoken  python-dotenv

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
os.environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')

In [ ]:
### Tracing (optional)
import os
from pprint import pprint

# os.environ['LANGCHAIN TRACING V2'] = 'true'
# os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
# os.environ['LANGCHAIN API KEY'] = userdata.get('LANGCHAIN_API_KEY')

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import PromptTemplate

from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

## 2. モデルをセット (Set the model throught API)

In [ ]:
from langchain_groq import ChatGroq

model = ChatGroq(model="llama3-70b-8192",temperature=0)

In [ ]:
#Categorize complaints
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are a Categorizer Agent You are a master at understanding what product do customers complaint of in the complaints? You can choose only one product.

     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Conduct a comprehensive analysis of the complaints provided and categorize into one of the following categories:
        Mortgage\
        Checking or savings account \
        Student loan\
        Money transfer, virtual currency, or money service\
        Bank account or service\
        Consumer Loan\

            Output a single cetgory only from the types ( 'Mortgage', 'Checking or savings account', 'Student loan', 'Money transfer, virtual currency, or money service',\
       'Bank account or service', 'Consumer Loan'. Your answer should be one of the name of product in the list only. Do not include other words. \) \
            eg:
            'Mortgage' \

    EMAIL CONTENT:\n\n {email} \n\n
    <|eot_id|>
    <|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["email"],
)

email_category_generator = prompt | model | StrOutputParser()

morg="Greetree ( changed name to Ditech servicing ) calls me all hours of the night. A representative name XXXX calls me. I told them that the only number the are allowed to contact me on is XXXX. My mother has said they also call on their number which is a separate line from my own line. Even after telling them this for months and years they still abuse this. As I have said before that the only line that I can be reach is on XXXX. I work from home and that is the only number that they can contact me regarding any information about the mortgage. I do n't know how clear I can be on discussing this with them regarding XXXX constant harassment. I am also considering legal action."

result = email_category_generator.invoke({"email": morg})

print(result)

Mortgage


In [ ]:
morg

"Greetree ( changed name to Ditech servicing ) calls me all hours of the night. A representative name XXXX calls me. I told them that the only number the are allowed to contact me on is XXXX. My mother has said they also call on their number which is a separate line from my own line. Even after telling them this for months and years they still abuse this. As I have said before that the only line that I can be reach is on XXXX. I work from home and that is the only number that they can contact me regarding any information about the mortgage. I do n't know how clear I can be on discussing this with them regarding XXXX constant harassment. I am also considering legal action."

## 3. dataをインポート (load data)


In [ ]:
import time
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, classification_report

In [ ]:
X=pd.read_csv("CC_20k_20230114.csv")
X

,Product,Issue,text,Complaint ID,Ja
0,Checking or savings account,Managing an account,XX/XX/2022. I began noticing charges to my deb...,6301154,xx/xx/2022。私は自分のデビットカードへの料金とオンラインウェルズファーゴアカウント...
1,Checking or savings account,Managing an account,I have a joint account with my Daughter. She g...,6280688,私は娘と共同アカウントを持っています。彼女はxxxx xxxxでxxxxに行き、私はxxxx...
2,Mortgage,Trouble during payment process,I made a principal payment of {$9900.00} on XX...,6257306,xx/xx/22で{$ 9900.00}の主要な支払いを行いました。米国銀行は、XXXX X...
3,Checking or savings account,Managing an account,I created a Savings account XX/XX/2022 with Al...,6246992,Ally Bankで普通預金口座XX/XX/2022を作成し、{$ 400.00}のデポジッ...
4,Mortgage,Trouble during payment process,Loan Number XXXX Reference : Mortgage XXXXXXXX...,6209018,ローン番号xxxxリファレンス：Mortgage xxxxxxx xxxx xxxx xxx...
...,...,...,...,...,...
199390,Bank account or service,"Account opening, closing, or management",Near the end of XXXX 2016 I opened a Citigold ...,2138969,XXXX 2016の終わり近くに、プロモーションコードXXXXでCitGold当座預金口座を...
199391,Student loan,Dealing with my lender or servicer,My loans have an extraordinarily high interest...,2302907,私のローンには非常に高い金利があり、私はそれらを借り換えることができません。
199392,Mortgage,"Loan modification,collection,foreclosure",Caliber Home Loans ( CHL ) sent confirmation i...,2302852,キャリバーホームローン（CHL）は、2016年のXXXXで確認を送信し、ローンの変更が承認さ...
199393,Consumer Loan,Managing the loan or lease,On XX/XX/XXXX I purchased a XXXX XXXX XXXX XXX...,2282405,xx/xx/xxxxでは、xxxx xxxx xxxx xxxxを購入しました。オハイオ州ド...


In [ ]:
from sklearn.preprocessing import LabelEncoder

lab = LabelEncoder()
lab = lab.fit(X["Product"])
X["label"] = lab.transform(X["Product"])
X

,Product,Issue,text,Complaint ID,Ja,label
0,Checking or savings account,Managing an account,XX/XX/2022. I began noticing charges to my deb...,6301154,xx/xx/2022。私は自分のデビットカードへの料金とオンラインウェルズファーゴアカウント...,1
1,Checking or savings account,Managing an account,I have a joint account with my Daughter. She g...,6280688,私は娘と共同アカウントを持っています。彼女はxxxx xxxxでxxxxに行き、私はxxxx...,1
2,Mortgage,Trouble during payment process,I made a principal payment of {$9900.00} on XX...,6257306,xx/xx/22で{$ 9900.00}の主要な支払いを行いました。米国銀行は、XXXX X...,4
3,Checking or savings account,Managing an account,I created a Savings account XX/XX/2022 with Al...,6246992,Ally Bankで普通預金口座XX/XX/2022を作成し、{$ 400.00}のデポジッ...,1
4,Mortgage,Trouble during payment process,Loan Number XXXX Reference : Mortgage XXXXXXXX...,6209018,ローン番号xxxxリファレンス：Mortgage xxxxxxx xxxx xxxx xxx...,4
...,...,...,...,...,...,...
199390,Bank account or service,"Account opening, closing, or management",Near the end of XXXX 2016 I opened a Citigold ...,2138969,XXXX 2016の終わり近くに、プロモーションコードXXXXでCitGold当座預金口座を...,0
199391,Student loan,Dealing with my lender or servicer,My loans have an extraordinarily high interest...,2302907,私のローンには非常に高い金利があり、私はそれらを借り換えることができません。,5
199392,Mortgage,"Loan modification,collection,foreclosure",Caliber Home Loans ( CHL ) sent confirmation i...,2302852,キャリバーホームローン（CHL）は、2016年のXXXXで確認を送信し、ローンの変更が承認さ...,4
199393,Consumer Loan,Managing the loan or lease,On XX/XX/XXXX I purchased a XXXX XXXX XXXX XXX...,2282405,xx/xx/xxxxでは、xxxx xxxx xxxx xxxxを購入しました。オハイオ州ド...,2


In [ ]:
Xp=X[0:5000]
Xp["Product"].value_counts()

Product
Checking or savings account                           2000
Mortgage                                              1660
Student loan                                           628
Money transfer, virtual currency, or money service     428
Bank account or service                                192
Consumer Loan                                           92
Name: count, dtype: int64

In [ ]:
X=X[5000:]
X

,Product,Issue,text,Complaint ID,Ja,label
5000,Consumer Loan,Managing the loan or lease,In XXXX we applied for a line of credit with a...,1630857,XXXXでは、一部の債務を統合するために、信用組合と信用枠を申請しました。彼らの検索は、私が...,2
5001,Checking or savings account,Managing an account,XX/XX/XXXX M & T bank called me and stated my ...,5958933,XX/XX/XXXX M＆T Bankが私に電話をかけ、私のアカウントが{$ 2000.00...,1
5002,"Money transfer, virtual currency, or money ser...",Fraud or scam,On XX/XX/XXXX I purchased a watch online and p...,5986714,xx/xx/xxxxでは、オンラインで時計を購入し、xxxxを使用して売り手に{$ 4500...,3
5003,Checking or savings account,Problem with a lender or other company chargin...,Title is exactly as stated. \n\nTD Bank back i...,6007929,タイトルはまさに述べられたとおりです。\n\nXXXXに戻ってTD銀行は、非行のために私の口...,1
5004,Mortgage,"Loan servicing, payments, escrow account",I refinanced my loan with Wells Fargo. It was ...,2309108,私はウェルズ・ファーゴとのローンを借り換えました。そもそも悪夢でしたが、最終的にXXXXで最...,4
...,...,...,...,...,...,...
199390,Bank account or service,"Account opening, closing, or management",Near the end of XXXX 2016 I opened a Citigold ...,2138969,XXXX 2016の終わり近くに、プロモーションコードXXXXでCitGold当座預金口座を...,0
199391,Student loan,Dealing with my lender or servicer,My loans have an extraordinarily high interest...,2302907,私のローンには非常に高い金利があり、私はそれらを借り換えることができません。,5
199392,Mortgage,"Loan modification,collection,foreclosure",Caliber Home Loans ( CHL ) sent confirmation i...,2302852,キャリバーホームローン（CHL）は、2016年のXXXXで確認を送信し、ローンの変更が承認さ...,4
199393,Consumer Loan,Managing the loan or lease,On XX/XX/XXXX I purchased a XXXX XXXX XXXX XXX...,2282405,xx/xx/xxxxでは、xxxx xxxx xxxx xxxxを購入しました。オハイオ州ド...,2


In [ ]:
X["Product"].value_counts()

Product
Mortgage                                              88101
Checking or savings account                           52372
Student loan                                          26787
Money transfer, virtual currency, or money service    13184
Bank account or service                                9546
Consumer Loan                                          4405
Name: count, dtype: int64

In [ ]:
Xt=X[["text"]]
Xt

,text
5000,In XXXX we applied for a line of credit with a...
5001,XX/XX/XXXX M & T bank called me and stated my ...
5002,On XX/XX/XXXX I purchased a watch online and p...
5003,Title is exactly as stated. \n\nTD Bank back i...
5004,I refinanced my loan with Wells Fargo. It was ...
...,...
199390,Near the end of XXXX 2016 I opened a Citigold ...
199391,My loans have an extraordinarily high interest...
199392,Caliber Home Loans ( CHL ) sent confirmation i...
199393,On XX/XX/XXXX I purchased a XXXX XXXX XXXX XXX...


In [ ]:
yl=X[["label"]]
yl

,label
5000,2
5001,1
5002,3
5003,1
5004,4
...,...
199390,0
199391,5
199392,4
199393,2


In [ ]:
max(yl["label"])

5

## 4. 訓練データと検証データを作成　(Prepare training and test data)


In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(Xt, yl, stratify=yl, train_size=0.013, test_size=0.013, random_state=100)
len(X_train), len(y_train), len(X_valid), len(y_valid)

(2527, 2527, 2528, 2528)

In [ ]:
X_valid

,text
85871,I was recently sold from XXXX XXXX XXXX to set...
113030,2nd Mortgage Servicer Planet Home Lending Acc ...
57238,XXXX XXXX is an elderly women living in her ho...
130633,My mortgage got transferred to Nationstar Mort...
26169,OCWEN company has big problems communicating i...
...,...
159445,The reason for this complaint is that i believ...
157764,we are currently delinquent on our mortgage an...
25321,This is a follow up to three prior complaints ...
120677,"Campus Partners, my loan servicer, has been co..."


In [ ]:
y_valid

,label
85871,4
113030,4
57238,4
130633,4
26169,4
...,...
159445,5
157764,4
25321,4
120677,5


In [ ]:
y_valid.nunique()

label    6
dtype: int64

In [ ]:
data = pd.concat([y_valid, X_valid], axis=1)
data

,label,text
85871,4,I was recently sold from XXXX XXXX XXXX to set...
113030,4,2nd Mortgage Servicer Planet Home Lending Acc ...
57238,4,XXXX XXXX is an elderly women living in her ho...
130633,4,My mortgage got transferred to Nationstar Mort...
26169,4,OCWEN company has big problems communicating i...
...,...,...
159445,5,The reason for this complaint is that i believ...
157764,4,we are currently delinquent on our mortgage an...
25321,4,This is a follow up to three prior complaints ...
120677,5,"Campus Partners, my loan servicer, has been co..."


## 5. Llama3-70Bによりクレームを判別 (Classify customer complaints by Llama3-70B)

In [ ]:
%%time

result=[]
n=0
m=500

for i in range(n,m):

  out = email_category_generator.invoke({"email": data.iloc[i,1]})
  result.append(out)

result

CPU times: user 9.53 s, sys: 1.11 s, total: 10.6 s
Wall time: 25min 52s


['Student loan',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Money transfer, virtual currency, or money service',
 'Bank account or service',
 'Money transfer, virtual currency, or money service',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Student loan',
 'Checking or savings account',
 'Mortgage',
 'Student loan',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Student loan',
 'Mortgage',
 'Consumer Loan',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Checking or savings account',
 'Mortgage',
 'Mortgage',
 '

In [ ]:
range(n,m)

range(0, 500)

In [ ]:
 list(lab.inverse_transform([0, 1, 2, 3, 4, 5]))

['Bank account or service',
 'Checking or savings account',
 'Consumer Loan',
 'Money transfer, virtual currency, or money service',
 'Mortgage',
 'Student loan']

In [ ]:
s=result.copy()
for i in range(0,m-n):
    if "Bank account or service" in s[i]:
      s[i]=0
    elif "Checking or savings account" in s[i]:
      s[i]=1
    elif "Consumer Loan" in s[i]:
      s[i]=2
    elif "Money transfer, virtual currency, or money service" in s[i]:
      s[i]=3
    elif "Mortgage" in s[i]:
      s[i]=4
    elif "Student loan" in s[i]:
      s[i]=5
    else:
      s[i]=6
#s

## 6. 精度検証　(Evaluate the model)

In [ ]:
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7c8b9bbb2fe0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7c8b9bbb24d0>, model_name='llama3-70b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [ ]:
label=list(y_valid['label'])[n:m]

print("result:", s)
print("label:", label)

result: [5, 4, 4, 4, 4, 4, 1, 4, 1, 4, 3, 0, 3, 1, 4, 4, 4, 4, 4, 1, 5, 1, 4, 5, 4, 4, 4, 4, 1, 4, 5, 4, 2, 1, 4, 1, 4, 4, 1, 4, 4, 4, 1, 4, 4, 1, 4, 1, 4, 4, 4, 1, 2, 4, 4, 4, 4, 5, 4, 4, 4, 0, 3, 5, 1, 1, 4, 1, 4, 4, 1, 3, 3, 1, 0, 1, 1, 1, 1, 1, 4, 4, 1, 4, 1, 4, 4, 1, 4, 1, 4, 4, 4, 4, 4, 5, 1, 4, 1, 5, 4, 4, 2, 4, 1, 4, 4, 1, 1, 4, 4, 4, 1, 4, 3, 5, 1, 4, 4, 4, 4, 5, 1, 4, 5, 0, 5, 1, 5, 3, 4, 4, 5, 1, 4, 2, 5, 1, 4, 1, 4, 4, 5, 4, 5, 4, 5, 1, 0, 3, 4, 4, 1, 4, 4, 4, 3, 1, 3, 4, 0, 1, 1, 1, 4, 5, 4, 3, 4, 3, 1, 4, 4, 3, 4, 4, 1, 4, 4, 1, 3, 5, 4, 5, 1, 4, 0, 4, 5, 1, 1, 1, 4, 4, 4, 3, 4, 0, 3, 1, 2, 4, 5, 3, 4, 5, 4, 4, 2, 4, 4, 4, 0, 4, 4, 5, 5, 4, 1, 4, 4, 1, 4, 4, 1, 1, 4, 3, 4, 4, 1, 4, 4, 4, 4, 1, 1, 4, 4, 4, 1, 1, 4, 1, 3, 5, 1, 4, 4, 5, 4, 1, 4, 4, 1, 1, 4, 3, 4, 2, 4, 1, 5, 2, 1, 5, 4, 4, 4, 4, 1, 4, 1, 4, 4, 3, 4, 4, 2, 4, 1, 4, 5, 5, 5, 1, 1, 1, 4, 1, 1, 1, 4, 4, 3, 4, 4, 4, 1, 2, 1, 4, 5, 1, 4, 2, 4, 5, 4, 4, 1, 1, 1, 4, 5, 4, 1, 4, 4, 4, 4, 4, 4, 4, 4, 1, 1, 1, 4, 5, 1

分布を確認 (Check the distribution of data)

In [ ]:
sdf=pd.Series(s).value_counts()
sdf.sort_index()

0     12
1    143
2     18
3     31
4    234
5     62
Name: count, dtype: int64

In [ ]:
labeldf=pd.Series(label).value_counts()
labeldf.sort_index()

0     26
1    119
2     16
3     38
4    232
5     69
Name: count, dtype: int64

In [ ]:
print('accuracy score: {0:0.4f}'.format(accuracy_score(label, s)))


accuracy score: 0.8860


In [ ]:
print(classification_report(label, s, digits=4))

              precision    recall  f1-score   support

           0     0.2500    0.1154    0.1579        26
           1     0.7762    0.9328    0.8473       119
           2     0.7778    0.8750    0.8235        16
           3     0.9032    0.7368    0.8116        38
           4     0.9658    0.9741    0.9700       232
           5     0.9839    0.8841    0.9313        69

    accuracy                         0.8860       500
   macro avg     0.7762    0.7530    0.7569       500
weighted avg     0.8752    0.8860    0.8765       500



### accuracy score
Test data : 500

- Llama3-70B : accuracy 0.886, time 25m52s      


Test data : 500 (Japanese)

- Llama3-70B : accuracy 0.828, time 46m17s    
